## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Hyeon Gu Kim

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes
import matplotlib.pyplot as plt

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [2]:
data = load_diabetes(as_frame=True)
df = pd.DataFrame(np.c_[data.data, data.target], columns=data.feature_names + ['target'])
df_X = df[data.feature_names]
s_y = df['target']

## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

In [3]:
def get_linear_regression_model( df_X, s_y ):
    # your code here
    df_X = pd.concat( [pd.DataFrame({'intercept':np.ones(len(df_X))}), df_X.reset_index(drop=True)], axis=1)
    [beta_hat, residuals, rank, s] = np.linalg.lstsq(df_X, s_y, rcond=-1)
    return beta_hat

In [4]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
beta_hat

array([ 4.18818425,  1.77890808,  0.74032569, -1.3506416 ,  0.14535984])

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [5]:
def partition_data( df_X, s_y, k ):
    dict_k_df_X = dict()
    dict_k_s_y = dict()
    # give each rows random int from 1~k
    df_X['dummy'] = np.random.randint(1, k+1, size=len(df_X))
    
    # seperate them with k
    for i in range(1, k+1): 
        fold_df = df_X[df_X['dummy']==i]
        lst = list(fold_df.columns)
        lst.pop(-1) # remove 'dummy' from the list
        dict_k_df_X[i] = fold_df[lst]
        dict_k_s_y[i] = s_y[df_X['dummy']==i]
    #or Prof Cahney's version:
    '''
    rand_idx = np.random.randint(1, k+1, size=len(df_X))
    for i in np.unique(rand_idx):
        dict_k_df_X[i] = dict_k_df_X[rand_indx == i]
        dict_k_s_y[i] = dict_k_s_y[rand_indx == i]
    '''
    return (dict_k_df_X, dict_k_s_y)

In [6]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5)

In [7]:
# Check fold sizes
total = 0
for i in range(1, 6):
    print("Fold {} length of dataframe is {} and length of series is {}".format(i, len(dict_k_df_X[i]), len(dict_k_s_y[i])))
    total += len(dict_k_df_X[i])
    
print("The sum of the number of elements in each fold is {} and there are {} rows in the original df".format(total, len(df_X)))

Fold 1 length of dataframe is 92 and length of series is 92
Fold 2 length of dataframe is 98 and length of series is 98
Fold 3 length of dataframe is 75 and length of series is 75
Fold 4 length of dataframe is 75 and length of series is 75
Fold 5 length of dataframe is 102 and length of series is 102
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [8]:
def get_mae( s_y, s_y_hat):
    # your code here
    return np.mean(abs(s_y - s_y_hat))

In [9]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [10]:
mae = np.array([])
for k in dict_k_df_X.keys():
    test_x = pd.concat( [pd.DataFrame({'intercept':np.ones(len(dict_k_df_X[k]))}), dict_k_df_X[k].reset_index(drop=True)], axis=1)
    s_y = dict_k_s_y[k]
    train_x = pd.DataFrame()
    train_y = pd.Series()
    
    # merge the train datasets
    lst = list(dict_k_df_X.keys())
    lst.remove(k)
    for i in lst:
        train_x = pd.concat([train_x, dict_k_df_X[i]], axis=0, ignore_index=True)
        train_y = pd.concat([train_y, dict_k_s_y[i]], axis=0, ignore_index=True)
        
    # build the model using the training data
    beta_hat = get_linear_regression_model(train_x, train_y)
    s_y_hat = np.dot(test_x, beta_hat)
    mae = np.append( mae, get_mae(s_y, s_y_hat) )  

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
# Prof Chaney's version:
'''
mae = np.array([])
for k in dict_k_df_X.keys():
    train_df_X = pd.DataFrame()
    train_s_y = pd.Series(dtype=float)
    for not_k in dict_k_df_X.keys():
        if not k != k:
            train_df_X = pd.concat([train_df_X, dict_k_df_X[not_k]])
            train_s_y = train_s_y.append(dict_k_s_y[not_k])
        test_df_X = dict_k_df_X[k]
        s_y_hat = dict_k_s_y[k]
        
        #get the model
        this_beta_hat = get_linear_regression_model( train_df_X, train_s_y )
        
        # Apply the model, adding the ones column first to the data set
        test_df_X = pd.concat( [pd.DataFrame({'intercept':np.ones(len(test_df_X))}, index=test_df_X.index), test_df_X], axis=1)
        s_y_hat = text_df_X.dot(this_beta_hat)
        
        mae = np.append( mae, get_mae(test_s_y, s_y_hat))
'''

In [11]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 40.76, the max MAE is 47.96, and the mean MAE is 44.22


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [12]:
data = load_iris(as_frame=True)
df = pd.DataFrame(np.c_[data.data, data.target], columns=data.feature_names + ['target'])
df_X = df[data.feature_names]
s_y = df['target']
s_y.value_counts()

2.0    50
1.0    50
0.0    50
Name: target, dtype: int64

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [13]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5)

# Check fold sizes
total = 0
for i in range(1, 6):
    print("Fold {} length of dataframe is {} and length of series is {}".format(i, len(dict_k_df_X[i]), len(dict_k_s_y[i])))
    total += len(dict_k_df_X[i])
    
print("The sum of the number of elements in each fold is {} and there are {} rows in the original df".format(total, len(df_X)))

Fold 1 length of dataframe is 27 and length of series is 27
Fold 2 length of dataframe is 38 and length of series is 38
Fold 3 length of dataframe is 27 and length of series is 27
Fold 4 length of dataframe is 30 and length of series is 30
Fold 5 length of dataframe is 28 and length of series is 28
The sum of the number of elements in each fold is 150 and there are 150 rows in the original df


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [14]:
def get_acc( s_1, s_2 ):
    s = (s_1 == s_2).value_counts()[True] #number of equal elements
    return s/len(s_1)
    

In [15]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [16]:
from sklearn import tree 

In [17]:
def inner_merge(dict_x, dict_y, lst, temp):
    train_x = pd.DataFrame()
    train_y = pd.Series()
    
    # merge the datasets
    for i in lst:
        if i != lst[temp]: #if not test dataset, merge the dataset
            train_x = pd.concat([train_x, dict_x[i]], axis=0, ignore_index=True)
            train_y = pd.concat([train_y, dict_y[i]], axis=0, ignore_index=True)
            
    return (train_x, train_y, dict_x[lst[temp]], dict_y[lst[temp]])

In [18]:
def outer_merge(dict_x, dict_y, lst):
    train_x = pd.DataFrame()
    train_y = pd.Series()
    
    # merge the datasets
    for i in lst:
        train_x = pd.concat([train_x, dict_x[i]], axis=0, ignore_index=True)
        train_y = pd.concat([train_y, dict_y[i]], axis=0, ignore_index=True)
    return (train_x, train_y)

In [19]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = np.array([])
for k in dict_k_df_X.keys():
    #create a list wihtout k
    lst = list(dict_k_df_X.keys())
    lst.remove(k)
    # Inner Loop
    for pos_min_impurity in possible_min_impurity_decrease:
        inner_acc = np.array([]) #empty list for new p.m.i.d
        mean_acc_lst = []
        # Inner loop cross validation code here (use 4 folds, where the fold does not include k)
        for temp in range(len(lst)):
            (train_x, train_y, test_x, test_y) = inner_merge(dict_k_df_X, dict_k_s_y, lst, temp) # inner CV
            clf = tree.DecisionTreeClassifier(criterion='gini', min_impurity_decrease=pos_min_impurity) #Creat Decision Tree
            clf = clf.fit(train_x, train_y)
            yhat = clf.predict(test_x)
            acc = get_acc(test_y, yhat)
            inner_acc = np.append(inner_acc, acc) 
            
        #print the mean of accuracy from the 4CV:
        mean_acc = round(np.mean(inner_acc),2)
        mean_acc_lst.append(mean_acc)
        print("Testing {} min impurity decrease \n\tAverage accuracy over 4 folds is {}".format(pos_min_impurity, mean_acc))
        
    # Use best min impurity decrease to train model
    best_pmid = max(mean_acc_lst)
    idx = mean_acc_lst.index(best_pmid)
    print("\nBest min impurity decrease is {}\n\n".format(possible_min_impurity_decrease[idx]))
    # Now train and test outer dataset
    (train_x, train_y) = outer_merge(dict_k_df_X, dict_k_s_y, lst) 
    out_clf = tree.DecisionTreeClassifier(criterion='gini', min_impurity_decrease = best_pmid) #Creat Decision Tree
    out_clf = out_clf.fit(train_x, train_y) #train tree using best pmid of the 4 folds
    out_yhat = out_clf.predict(dict_k_df_X[k]) #use test data (kth) to see how the tree does
    # outer accuracy calculation 
    this_acc = get_acc(dict_k_s_y[k], out_yhat)
    outer_acc = np.append(outer_acc,this_acc) # make sure and calculate this_acc in your loop

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel

Testing 0.1 min impurity decrease 
	Average accuracy over 4 folds is 0.92
Testing 0.25 min impurity decrease 
	Average accuracy over 4 folds is 0.75
Testing 0.3 min impurity decrease 
	Average accuracy over 4 folds is 0.69
Testing 0.4 min impurity decrease 
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel

Testing 0.1 min impurity decrease 
	Average accuracy over 4 folds is 0.92
Testing 0.25 min impurity decrease 
	Average accuracy over 4 folds is 0.76
Testing 0.3 min impurity decrease 
	Average accuracy over 4 folds is 0.64
Testing 0.4 min impurity decrease 
	Average accuracy over 4 folds is 0.29

Best min impurity decrease is 0.1




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel

Testing 0.1 min impurity decrease 
	Average accuracy over 4 folds is 0.96
Testing 0.25 min impurity decrease 
	Average accuracy over 4 folds is 0.96
Testing 0.3 min impurity decrease 
	Average accuracy over 4 folds is 0.71
Testing 0.4 min impurity decrease 
	Average accuracy over 4 folds is 0.29

Best min impurity decrease is 0.1




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel

Testing 0.1 min impurity decrease 
	Average accuracy over 4 folds is 0.91
Testing 0.25 min impurity decrease 
	Average accuracy over 4 folds is 0.77
Testing 0.3 min impurity decrease 
	Average accuracy over 4 folds is 0.64
Testing 0.4 min impurity decrease 
	Average accuracy over 4 folds is 0.31

Best min impurity decrease is 0.1


Testing 0.1 min impurity decrease 
	Average accuracy over 4 folds is 0.96
Testing 0.25 min impurity decrease 
	Average accuracy over 4 folds is 0.96
Testing 0.3 min impurity decrease 
	Average accuracy over 4 folds is 0.82
Testing 0.4 min impurity decrease 
	Average accuracy over 4 folds is 0.36

Best min impurity decrease is 0.1




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel

In [20]:
outer_acc

array([0.66666667, 0.63157895, 0.62962963, 0.6       , 0.21428571])

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [21]:
print("General Performance of the Classifier")
print("Min: {}".format(np.amin(outer_acc)))
print("Max: {}".format(np.amax(outer_acc)))
print("Mean Accuracy: {}".format(np.sum(outer_acc)/len(outer_acc)))

General Performance of the Classifier
Min: 0.21428571428571427
Max: 0.6666666666666666
Mean Accuracy: 0.5484321915900864
